In [1]:
import cv2
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import glob
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

# Helper function to load images
def load_images(path):
    images = []
    labels = []
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            img = cv2.imread(file)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(1 if label == 'Positive' else 0)
    return np.array(images), np.array(labels)

# Load datasets
train_images, train_labels = load_images(train_path)
test_images, test_labels = load_images(test_path)
val_images, val_labels = load_images(val_path)

# Function to build the graph representation of the image
def build_graph(image):
    graph = nx.Graph()
    h, w, _ = image.shape
    for i in range(h):
        for j in range(w):
            graph.add_node((i, j), pixel=image[i, j])
            if i > 0:
                graph.add_edge((i, j), (i-1, j), weight=np.linalg.norm(image[i, j] - image[i-1, j]))
            if j > 0:
                graph.add_edge((i, j), (i, j-1), weight=np.linalg.norm(image[i, j] - image[i, j-1]))
    return graph

# Function to visualize the graph
def visualize_graph(graph, title):
    pos = {node: node for node in graph.nodes()}
    weights = [graph[u][v]['weight'] for u, v in graph.edges()]
    plt.figure(figsize=(10, 10))
    nx.draw(graph, pos, node_size=5, edge_color=weights, edge_cmap=plt.cm.Blues)
    plt.title(title)
    plt.show()

# Function to extract features from the graph
def extract_features(image):
    graph = build_graph(image)
    features = []
    for node in graph.nodes(data=True):
        features.extend(node[1]['pixel'])
    return np.array(features)

# Handling class imbalance using SMOTE
smote = SMOTE(random_state=42)

# Extract features for all images
X_train = np.array([extract_features(img) for img in train_images])
X_test = np.array([extract_features(img) for img in test_images])
X_val = np.array([extract_features(img) for img in val_images])

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(train_labels)
y_test = le.transform(test_labels)
y_val = le.transform(val_labels)

# Apply SMOTE to balance the classes
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Evaluate the classifier
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# Visualize the graphs for a sample positive and negative image
positive_image = None
negative_image = None
for img, label in zip(train_images, train_labels):
    if label == 1 and positive_image is None:
        positive_image = img
    elif label == 0 and negative_image is None:
        negative_image = img
    if positive_image is not None and negative_image is not None:
        break

# Build graphs for the sample images
positive_graph = build_graph(positive_image)
negative_graph = build_graph(negative_image)

# Visualize the graphs
visualize_graph(positive_graph, 'Graph Representation of Positive Image (Lyme Disease)')
visualize_graph(negative_graph, 'Graph Representation of Negative Image (No Lyme Disease)')


              precision    recall  f1-score   support

    Negative       0.83      0.86      0.84       412
    Positive       0.26      0.22      0.24        95

    accuracy                           0.74       507
   macro avg       0.54      0.54      0.54       507
weighted avg       0.72      0.74      0.73       507



TypeError: '_AxesStack' object is not callable

<Figure size 1000x1000 with 0 Axes>

In [6]:
import cv2
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import glob
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.feature import local_binary_pattern
from keras.preprocessing.image import ImageDataGenerator

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

# Helper function to load images
def load_images(path):
    images = []
    labels = []
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            img = cv2.imread(file)
            if img is None:
                print(f"Warning: Unable to read image {file}")
                continue
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(1 if label == 'Positive' else 0)
    return np.array(images), np.array(labels)

# Load datasets
train_images, train_labels = load_images(train_path)
test_images, test_labels = load_images(test_path)
val_images, val_labels = load_images(val_path)

# Function to build the graph representation of the image
def build_graph(image):
    graph = nx.Graph()
    h, w, _ = image.shape
    for i in range(h):
        for j in range(w):
            graph.add_node((i, j), pixel=image[i, j])
            if i > 0:
                graph.add_edge((i, j), (i-1, j), weight=np.linalg.norm(image[i, j] - image[i-1, j]))
            if j > 0:
                graph.add_edge((i, j), (i, j-1), weight=np.linalg.norm(image[i, j] - image[i, j-1]))
    return graph

# Function to visualize the graph
def visualize_graph(graph, title):
    pos = {node: node for node in graph.nodes()}
    weights = [graph[u][v]['weight'] for u, v in graph.edges()]
    plt.figure(figsize=(10, 10))
    nx.draw(graph, pos, node_size=5, edge_color=weights, edge_cmap=plt.cm.Blues)
    plt.title(title)
    plt.show()

# Advanced feature extraction using Local Binary Patterns (LBP)
def extract_features(image):
    lbp = local_binary_pattern(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Apply data augmentation to the training set
augmented_images = []
augmented_labels = []

for img, label in zip(train_images, train_labels):
    img = img.reshape((1,) + img.shape)
    i = 0
    for batch in datagen.flow(img, batch_size=1):
        augmented_images.append(batch[0])
        augmented_labels.append(label)
        i += 1
        if i >= 5:  # Number of augmented samples per image
            break

train_images = np.array(augmented_images)
train_labels = np.array(augmented_labels)

# Handling class imbalance using SMOTE
smote = SMOTE(random_state=42)

# Extract features for all images
X_train = np.array([extract_features(img) for img in train_images])
X_test = np.array([extract_features(img) for img in test_images])
X_val = np.array([extract_features(img) for img in val_images])

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(train_labels)
y_test = le.transform(test_labels)
y_val = le.transform(val_labels)

# Apply SMOTE to balance the classes
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train a classifier
clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Evaluate the classifier
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

# Visualize the graphs for a sample positive and negative image
positive_image = None
negative_image = None
for img, label in zip(train_images, train_labels):
    if label == 1 and positive_image is None:
        positive_image = img
    elif label == 0 and negative_image is None:
        negative_image = img
    if positive_image is not None and negative_image is not None:
        break

# Check if images were found
if positive_image is not None and negative_image is not None:
    # Build graphs for the sample images
    positive_graph = build_graph(positive_image)
    negative_graph = build_graph(negative_image)

    # Visualize the graphs
    visualize_graph(positive_graph, 'Graph Representation of Positive Image (Lyme Disease)')
    visualize_graph(negative_graph, 'Graph Representation of Negative Image (No Lyme Disease)')
else:
    print("Sample images for positive and negative classes not found.")



              precision    recall  f1-score   support

    Negative       0.81      0.94      0.87       412
    Positive       0.08      0.02      0.03        95

    accuracy                           0.77       507
   macro avg       0.44      0.48      0.45       507
weighted avg       0.67      0.77      0.71       507



TypeError: '_AxesStack' object is not callable

<Figure size 1000x1000 with 0 Axes>

In [7]:
from sklearn.linear_model import LogisticRegression

# Train a Logistic Regression model
clf_lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
clf_lr.fit(X_train_resampled, y_train_resampled)

# Evaluate the Logistic Regression model
y_pred_lr = clf_lr.predict(X_test)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Negative', 'Positive']))


Logistic Regression Classification Report:
              precision    recall  f1-score   support

    Negative       0.84      0.57      0.68       412
    Positive       0.22      0.53      0.31        95

    accuracy                           0.56       507
   macro avg       0.53      0.55      0.49       507
weighted avg       0.72      0.56      0.61       507



In [8]:
from sklearn.svm import SVC

# Train a Support Vector Machine model
clf_svm = SVC(class_weight='balanced', random_state=42)
clf_svm.fit(X_train_resampled, y_train_resampled)

# Evaluate the SVM model
y_pred_svm = clf_svm.predict(X_test)
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm, target_names=['Negative', 'Positive']))


SVM Classification Report:
              precision    recall  f1-score   support

    Negative       0.85      0.61      0.71       412
    Positive       0.23      0.52      0.32        95

    accuracy                           0.59       507
   macro avg       0.54      0.56      0.51       507
weighted avg       0.73      0.59      0.64       507



In [1]:
import cv2
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import glob
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

# Helper function to load images
def load_images(path):
    images = []
    labels = []
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            img = cv2.imread(file)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(1 if label == 'Positive' else 0)
    return np.array(images), np.array(labels)

# Load datasets
train_images, train_labels = load_images(train_path)
test_images, test_labels = load_images(test_path)
val_images, val_labels = load_images(val_path)

# Function to build the graph representation of the image
def build_graph(image):
    graph = nx.Graph()
    h, w, _ = image.shape
    for i in range(h):
        for j in range(w):
            graph.add_node((i, j), pixel=image[i, j])
            if i > 0:
                graph.add_edge((i, j), (i-1, j), weight=np.linalg.norm(image[i, j] - image[i-1, j]))
            if j > 0:
                graph.add_edge((i, j), (i, j-1), weight=np.linalg.norm(image[i, j] - image[i, j-1]))
    return graph

# Function to visualize the graph
def visualize_graph(graph, title):
    pos = {node: node for node in graph.nodes()}
    weights = [graph[u][v]['weight'] for u, v in graph.edges()]
    plt.figure(figsize=(10, 10))
    nx.draw(graph, pos, node_size=5, edge_color=weights, edge_cmap=plt.cm.Blues)
    plt.title(title)
    plt.show()

# Function to extract features from the graph
def extract_features(image):
    graph = build_graph(image)
    features = []
    for node in graph.nodes(data=True):
        features.extend(node[1]['pixel'])
    return np.array(features)

# Handling class imbalance using SMOTE
smote = SMOTE(random_state=42)

# Extract features for all images
X_train = np.array([extract_features(img) for img in train_images])
X_test = np.array([extract_features(img) for img in test_images])
X_val = np.array([extract_features(img) for img in val_images])

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(train_labels)
y_test = le.transform(test_labels)
y_val = le.transform(val_labels)

# Apply SMOTE to balance the classes
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [2]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, scoring='f1', verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train_resampled, y_train_resampled)

# Best parameters and score
print("Best Parameters:")
print(grid_search.best_params_)
print("Best Score:")
print(grid_search.best_score_)

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print("Best SVM Classification Report:")
print(classification_report(y_test, y_pred_best, target_names=['Negative', 'Positive']))


NameError: name 'SVC' is not defined

In [ ]:

import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Assuming X_train_resampled, y_train_resampled, X_test, and y_test are already defined

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, scoring='f1', verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train_resampled, y_train_resampled)

# Best parameters and score
print("Best Parameters:")
print(grid_search.best_params_)
print("Best Score:")
print(grid_search.best_score_)

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print("Best SVM Classification Report:")
print(classification_report(y_test, y_pred_best, target_names=['Negative', 'Positive']))


Fitting 5 folds for each of 48 candidates, totalling 240 fits
